In [1]:
from prophet import Prophet

In [2]:
import pandas as pd
df = pd.read_csv('result2021.csv')
df

,No,全国地方公共団体コード,都道府県名,市区町村名,公表_年月日,発症_年月日,確定_年月日,患者_居住地,患者_年代,患者_性別,患者_職業,患者_状態,患者_症状,患者_渡航歴の有無フラグ,患者_接触歴の有無フラグ,備考,退院済フラグ
0,60132,130001,東京都,NaN,2021-01-01,2020-12-27,2020-12-31,都内,10代,男性,学生,NaN,NaN,NaN,NaN,NaN,1.0
1,60133,130001,東京都,NaN,2021-01-01,NaN,2020-12-27,都内,40代,男性,－,NaN,NaN,NaN,NaN,NaN,1.0
2,60134,130001,東京都,NaN,2021-01-01,2020-12-24,2020-12-29,都内,10代,女性,－,NaN,NaN,NaN,NaN,NaN,1.0
3,60135,130001,東京都,NaN,2021-01-01,NaN,2020-12-28,都内,50代,男性,－,NaN,NaN,NaN,NaN,NaN,1.0
4,60136,130001,東京都,NaN,2021-01-01,2020-12-29,2020-12-31,都内,40代,女性,－,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322743,378187,130001,東京都,NaN,2021-12-31,NaN,2021-12-30,都内,70代,男性,－,NaN,NaN,NaN,NaN,NaN,1.0
322744,378188,130001,東京都,NaN,2021-12-31,2021-12-26,2021-12-29,都内,20代,男性,接客業,NaN,NaN,NaN,NaN,NaN,1.0
322745,378189,130001,東京都,NaN,2021-12-31,2021-12-24,2021-12-27,都内,20代,女性,医療従事者,NaN,NaN,NaN,NaN,NaN,1.0
322746,378190,130001,東京都,NaN,2021-12-31,2021-12-21,2021-12-28,都内,50代,女性,主婦,NaN,NaN,NaN,NaN,NaN,1.0


In [3]:
df1 = df.groupby(["公表_年月日"]).count()["都道府県名"].reset_index()
df1 = df1.rename(index = str,columns={'公表_年月日': '日付'})
df1 = df1.rename(index = str,columns={'都道府県名': '人数'})
df1

,日付,人数
0,2021-01-01,793
1,2021-01-02,829
2,2021-01-03,826
3,2021-01-04,905
4,2021-01-05,1315
...,...,...
360,2021-12-27,35
361,2021-12-28,45
362,2021-12-29,76
363,2021-12-30,63


In [4]:
new_df = df1.rename(columns={'日付': 'ds', '人数': 'y'})
train_df = new_df
train_df.tail()

,ds,y
360,2021-12-27,35
361,2021-12-28,45
362,2021-12-29,76
363,2021-12-30,63
364,2021-12-31,78


In [5]:
event_dataframe = pd.read_csv('event1.csv')
event_dataframe.tail()

,Unnamed: 0,holiday,ds
62,62,文化の日,2022-11-03
63,63,勤労感謝の日,2022-11-23
64,64,クリスマスイヴ,2022-12-24
65,65,クリスマス,2022-12-25
66,66,大晦日,2022-12-31


In [6]:
# モデルの作成
model = Prophet(
    growth='logistic', # 傾向変動の関数．非線形は'logistic'一般的なビジネス/金融データは非線形なのでlogistic。
    yearly_seasonality = True, # 年次の季節変動を考慮有無
    weekly_seasonality = True, # 週次の季節変動を考慮有無
    daily_seasonality = False, # 日次の季節変動を考慮有無
    changepoints = None, #  傾向変化点のリスト 
    changepoint_range = 1, # 傾向変化点の候補の幅で先頭からの割合。
    changepoint_prior_scale = 0.5, # 傾向変化点の事前分布のスケール値。パラメータの柔軟性 トレンドの変化がオーバーフィット（柔軟性が高すぎる）またはアンダーフィット（柔軟性が不十分）の場合、 入力引数を使用する前にスパースの強度を調整できます。
    n_changepoints = 5, # 傾向変化点の数 changepointの場所を手動で設定したい場合これを使う Changepointとは、データ中で「トレンドが大きく変わったと思われる」点のことです。 changepointは自動検出されるが、手動でカスタマイズできる。 n_changepointsではトレンドの変更点の数を手動で設定できる。デフォルトでは25。
    holidays= event_dataframe
) 
train_df['cap'] = 7000 # logisticモデルを選択する場合は必ずcapの設定が必須。最大値は'cap'、最小値は'floor'で指定できる

model.fit(train_df)

14:47:16 - cmdstanpy - INFO - Chain [1] start processing
14:47:16 - cmdstanpy - INFO - Chain [1] done processing


In [7]:
predict_num = 267
future = model.make_future_dataframe(periods = predict_num ,freq='d')
future.tail()

,ds
627,2022-09-20
628,2022-09-21
629,2022-09-22
630,2022-09-23
631,2022-09-24


In [8]:
future['cap'] = 7000
future['floor'] = 0
forecast = model.predict(future)
forecast.tail(267)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
forecast.loc[forecast['yhat'] < 0, 'yhat'] = 0
forecast['yhat']

In [ ]:
import matplotlib.pyplot as plt
from prophet.plot import add_changepoints_to_plot
forecast['cap'] = 7000
forecast['floor'] = 0
fig1 = model.plot(forecast)
a = add_changepoints_to_plot(fig1.gca(), model, forecast)
plt.show()

In [ ]:
z = pd.read_csv('result1.csv')

In [ ]:
z = z.rename(columns={'日付': 'ds', '人数': 'y'})
z

In [ ]:
import numpy as np

In [ ]:
y = forecast
np.mean(abs(z['y'] - y['yhat']))